In [2]:
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
# nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize
# nltk.download()
from sklearn.model_selection import GridSearchCV
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import pickle
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix,plot_confusion_matrix

In [3]:
df_train = pd.read_csv("train.txt",delimiter=';',names=['text','label'])
df_val = pd.read_csv("val.txt",delimiter=';',names=['text','label'])
df = pd.concat([df_train,df_val])

df.reset_index(drop=True,inplace=True)
df['label'].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [4]:
# data=pd.read_csv("custom_dataset_for_train.csv")
# data

In [5]:
df_train

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [6]:
def enco(df):
    df.replace(to_replace="surprise",value=1,inplace=True)
    df.replace(to_replace="joy",value=2,inplace=True)
    df.replace(to_replace="love",value=2,inplace=True)
    df.replace(to_replace="sadness",value=3,inplace=True)
    df.replace(to_replace="anger",value=3,inplace=True)
    df.replace(to_replace="fear",value=1,inplace=True)
    
enco(df.label)
enco(df_train)
df.to_csv("df_enco.csv")
df_train.to_csv("df_enco_train.csv")

PermissionError: [Errno 13] Permission denied: 'df_enco.csv'

In [ ]:
pd1=pd.read_csv("df_enco.csv")
pd1

In [ ]:
fina=[]
def conver(data):
    for val in range(len(data)):
        lm = WordNetLemmatizer()
        stopword=set(stopwords.words("english"))
        wor=re.sub("[^A-Za-z]",' ',str(data[val]))
        wor=wor.lower() 
        wor=wor.split()
        wor = [lm.lemmatize(word) for word in wor if word not in set(stopwords.words('english'))]    
        tex=[]
        for ijx in wor:
            if ijx not in stopword:
                tex.append(lm.lemmatize(ijx))
        fina.append(" ".join(tex))
    print(fina)
    print(len(fina))    
    
train_fina=[]
def train_conver(data):
    for val in range(len(data)):
        lm = WordNetLemmatizer()
        stopword=set(stopwords.words("english"))
        wor=re.sub("[^A-Za-z]",' ',str(data[val]))
        wor=wor.lower() 
        wor=wor.split()
        wor = [lm.lemmatize(word) for word in wor if word not in set(stopwords.words('english'))]    
        tex=[]
        for ijx in wor:
            if ijx not in stopword:
                tex.append(lm.lemmatize(ijx))
        train_fina.append(" ".join(tex))
    print(train_fina)
    print(len(train_fina))
    

    

In [ ]:
# conver(df.text)
# train_conver(df_train.text)

# fina_data=pd.DataFrame(fina,columns=["process"])
# fina_data.to_csv("processed.csv")

# train_da=pd.DataFrame(train_fina,columns=["process"])
# train_da.to_csv("train_process.csv")

In [ ]:
process_data=pd.read_csv("processed.csv")
process_data["process"]
train_process=pd.read_csv("train_process.csv")
# train_process=pd.read_csv("custom_dattt.csv")
train_process["process"]
process_data["process"]

In [ ]:
vector=CountVectorizer()
x=vector.fit_transform(process_data["process"])
test=vector.transform(train_process["process"])
x.toarray()
# test.toarray()



In [ ]:
param={'n_estimators':[100,200,300],
      'criterion':['gini','entropy'],
#       'max_depth':[10,20,30,40,50,60,70,80,90,100],
#        'max_features':['auto','log2','sqrt'],
#        'min_sample_leaf':[3,5,7,9,11],
#        'min_samples_split': [2,5,10]
      }
# ,13,15,17,19,21,23,25 min sample

In [ ]:
model=RandomForestClassifier()
grid=GridSearchCV(model,param,verbose=5,cv=10)


In [ ]:
# # grid.fit(x,df.label)  
# grid.fit(x,pd.label) 

In [ ]:
# pickle.dump(grid, open("grid_model_with_2_hp_22.pkl", "wb"))
# 
# gridd = pickle.load(open("grid_model_with_2_hp_22.pkl", "rb"))

# pickle.dump(grid, open("grid_model_with_2_hp_22_v2.pkl", "wb"))
# 
gridd = pickle.load(open("grid_model_with_2_hp_22_v2.sav", "rb"))

# pickle.dump(grid, open("grid_model_with_2_hp_22_v2.sav", "wb"))

# pickle.dump(vector, open("vector.sav", "wb"))



In [ ]:
gridd

In [ ]:
print(gridd.cv_results_)
print(gridd.best_estimator_)
# print(gridd.best_index_)
print(gridd.best_params_)
print(gridd.best_score_)

In [ ]:
predic_grid_random=gridd.predict(test)
acc_grid_random=accuracy_score(df_train.label,predic_grid_random)
acc_grid_random


In [ ]:
#Confusion Matrix

result=confusion_matrix(y_true=df_train.label,y_pred=predic_grid_random)
print(result)
True_Positive =result[0][0]
False_Negative=result[0][1]+result[0][2]
False_Positive=result[1][0]+result[2][0]
True_Negative=result[1][1]+result[2][2]
print(f"True Positive :{True_Positive}")
print(f"False Negative :{False_Negative}")
print(f"False Positive :{False_Positive}")
print(f"True Negative :{True_Negative}")
matrix=[[True_Positive,False_Negative],[False_Positive,True_Negative]]
print(np.matrix(matrix))


In [ ]:
inp=input("Enter")
# gridd.predict(test)

train_fina=[]
# for val in range(len(df_train.text)):
lm = WordNetLemmatizer()
stopword=set(stopwords.words("english"))
wor=re.sub("[^A-Za-z]",' ',str(inp))
wor=wor.lower()    
wor=wor.split()
wor = [lm.lemmatize(word) for word in wor if word not in set(stopwords.words('english'))]
tex=[]
for ijx in wor:
    if ijx not in stopword:
        tex.append(lm.lemmatize(ijx))
train_fina.append(" ".join(tex))
print(train_fina)
print(len(train_fina))        
test=vector.transform(train_fina)
predic=gridd.predict(test)
# predic[0]

# if(predic[0] == 1):
#     print("Surprice")
    
# if(predic[0] == 2):
#     print("Joy")
# if(predic[0] == 3):
#     print("Love")
# if(predic[0] == 4):
#     print("Sadness")
# if(predic[0] == 5):
#     print("Anger")
# if(predic[0] == 6):
#     print("Fear")


# nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

In [ ]:


def check(inpp):
    train_fina=[]
    # for val in range(len(df_train.text)):
    lm = WordNetLemmatizer()
    stopword=set(stopwords.words("english"))
    wor=re.sub("[^A-Za-z]",' ',str(inpp))
    wor=wor.lower()    
    wor=wor.split()
    wor = [lm.lemmatize(word) for word in wor if word not in set(stopwords.words('english'))]
    tex=[]
    for ijx in wor:
        if ijx not in stopword:
            tex.append(lm.lemmatize(ijx))
    train_fina.append(" ".join(tex))
    print(train_fina)
    print(len(train_fina))        
    test=vector.transform(train_fina)
    predic=gridd.predict(test)
    predic[0]

    if(predic[0] == 1):
        print("Neutral")

    if(predic[0] == 2):
        print("Positive")
    if(predic[0] == 3):
        print("Negative")
    # if(predic[0] == 4):
    #     print("Sadness")
    # if(predic[0] == 5):
    #     print("Anger")
    # if(predic[0] == 6):
    #     print("Fear")

    # elif(predic[0] == 0):
    #     print("sadness, anger, fear")

inp=input()
check(inp)